In [1]:
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
files=["../../feature_extraction/concatenated_data/cnn_human.txt","../../feature_extraction/concatenated_data/cnn_continue.txt"]

file1 = open(files[0], "r")
text_human = file1.read()

file2 = open(files[1], "r")
text_continue = file2.read()


In [32]:
import torch
import transformers
import scipy as sp

In [33]:
# load a BERT sentiment analysis model
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)
model = transformers.DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
).cuda()

In [34]:
# define a prediction function
def f(x):
    tv = torch.tensor(
        [
            tokenizer.encode(v, padding="max_length", max_length=500, truncation=True)
            for v in x
        ]
    ).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
    return val

In [35]:
# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)

In [ ]:
text_human = " ".join(text_human.split()[:1000])
text_human
text_continue = text_continue[:1000]
text_continue

In [55]:
shap_values = explainer(text_continue, fixed_context=1)

Partition explainer: 1001it [00:32, 21.10it/s]                          


In [57]:
# plot the first sentence's explanation
shap.plots.text(shap_values[:10])

In [46]:
# plot the first sentence's explanation
shap.plots.text(shap_values[:3])
